In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
import merge_data

In [4]:
DIGEST_SEGMENT_SPEEDS = GTFS_DATA_DICT.digest_tables.route_segment_speeds

In [5]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [6]:
analysis_date_list = (
        rt_dates.y2024_dates + rt_dates.y2023_dates
    )
    

In [7]:
DIGEST_SEGMENT_SPEEDS

'digest/segment_speeds'

In [8]:
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [9]:
primary_typology = merge_data.set_primary_typology(df_sched)

In [10]:
segment_speeds23 = gpd.read_parquet("2023_seg_speeds.parquet")

In [11]:
segment_speeds24 = gpd.read_parquet("2024_seg_speeds.parquet")

In [12]:
segment_speeds = pd.concat([segment_speeds23,segment_speeds24])

In [13]:
segment_speeds.shape

(4933047, 15)

In [14]:
primary_typology.shape

(27868, 5)

In [15]:
segment_speeds2 = pd.merge(
        segment_speeds,
        primary_typology,
        on = merge_data.route_time_cols,
        how = "left"
    )

In [16]:
segment_speeds2.shape

(4933047, 16)

In [17]:
type(segment_speeds2)

geopandas.geodataframe.GeoDataFrame

In [18]:
from shared_utils import gtfs_utils_v2, publish_utils

In [19]:
public_feeds = gtfs_utils_v2.filter_to_public_schedule_gtfs_dataset_keys()

In [20]:
 publish_utils.exclude_private_datasets??

Signature:
publish_utils.exclude_private_datasets(
    df: pandas.core.frame.DataFrame,
    col: str = 'schedule_gtfs_dataset_key',
    public_gtfs_dataset_keys: list = [],
) -> pandas.core.frame.DataFrame
Source:   
def exclude_private_datasets(
    df: pd.DataFrame,
    col: str = "schedule_gtfs_dataset_key",
    public_gtfs_dataset_keys: list = [],
) -> pd.DataFrame:
    """
    Filter out private datasets.
    """
    return df[df[col].isin(public_gtfs_dataset_keys)].reset_index(drop=True)
File:      ~/data-analyses/_shared_utils/shared_utils/publish_utils.py
Type:      function

In [ ]:
segment_speeds2.pipe(
        publish_utils.exclude_private_datasets, 
        public_gtfs_dataset_keys= public_feeds)

In [ ]:
# segment_speeds2.to_parquet("segment_speeds2.parquet")

In [ ]:
df = pd.read_parquet(f"{RT_SCHED_GCS}{DIGEST_SEGMENT_SPEEDS}.parquet")

In [ ]:
df.shape

In [ ]:
primary_typology.shape

In [ ]:
dates_2024 = rt_dates.y2024_dates

In [ ]:
segment_speeds_2024 = merge_data.concatenate_segment_speeds_by_route_direction(
        dates_2024
    ).pipe(
        merge_data.merge_in_standardized_route_names, 
    ).astype({"direction_id": "int64"}) #Int64 doesn't work for gdf

In [ ]:
segment_speeds_2024.to_parquet(f"2024_seg_speeds.parquet")

In [ ]:
dates_2023 = rt_dates.y2023_dates

In [ ]:
segment_speeds_2023 = merge_data.concatenate_segment_speeds_by_route_direction(
        dates_2023
    ).pipe(
        merge_data.merge_in_standardized_route_names, 
    ).astype({"direction_id": "int64"}) #Int64 doesn't work for gdf

In [ ]:
segment_speeds_2023.to_parquet(f"2023_seg_speeds.parquet")

In [ ]:
segment_speeds = pd.concat([segment_speeds_2024, segment_speeds_2023])

In [ ]:
segment_speeds.to_parquet(f"all_seg_speeds.parquet")